### Rename dict.keys() 

"NLI result" -> "NLI score"

In [3]:
import utils

inp_file = "./../exp/trivia/evaluate/val_2k_en_pred_eval.json"
out_file = "./../exp/trivia/evaluate/val_2k_en_pred_eval_temp.json"

data_pool = utils.read_json(inp_file)

for data in data_pool:
    data["NLI result"] = data.pop("NLI score")

utils.write_json(out_file, data_pool)

### Reformat jsonlines to json.

In [12]:
import json
import utils

inp_file = "./../exp/trivia/evaluate/val_2k_zh_pred_eval.json"
out_file = "./../exp/trivia/evaluate/val_2k_zh_pred_eval_temp.json"

with open(inp_file, "r") as fr:
    data_pool = [json.loads(line) for line in fr.readlines()]

utils.write_json(out_file, data_pool)

### Adding answers in eval files.

In [16]:
import utils

lang = "ja"
eval_file = "./../exp/trivia/evaluate/val_2k_{}_pred_eval_en.json".format(lang)
data_file = "./../exp/trivia/generate/val_2k_{}_gpt-3.5-turbo_T0.8.json".format(lang)

data_pool, eval_pool = utils.read_json(data_file), utils.read_json(eval_file)

for idx, samp in enumerate(eval_pool):
    assert samp["question_id"] == data_pool[idx]["question_id"]
    samp["generated answer"] = data_pool[idx]["prediction"]
    samp["gold answer"] = data_pool[idx]["answer"]

utils.write_json(eval_file, eval_pool)

### ChatGPT return value tests

In [1]:
import utils

model_name = "gpt-3.5-turbo"
temperature = 1.0
input_context = "# Question #: What's the captial city of France?\n# Proposed Answer #: Paris\n\n" \
                "Is the proposed answer:\nTrue\nFalse\nThe possible answer is: "


response = utils.get_chatgpt_info(model_name, input_context, temperature)

print(response)

PermissionError: Country, region, or territory not supported

In [6]:
import os
import json
import random
from para import lang_map

data_dir = "../data/{}/mling_{}.json"
output_path = "../data/human/en_zh_{}_samples.json"
datasets = ["common", "trivia", "sciq", "gsm8k"]
language_list = ["Japanese", "French", "Thai"]



for language in language_list:
    language = lang_map[language]
    sample_list = []
    for dataset in datasets:
        data_path = data_dir.format(dataset, dataset)
        print(dataset, data_path)
        with open(data_path, "r") as fr:
            data_pool = json.load(fr)
        data_pool = random.choices(data_pool, 20)
        for data in data_pool:
            sample = {
                "question_id": data["question_id"],
                "question": {
                    "en": data["en"]["question"],
                    "zh": data["zh"]["question"],
                    language: data[language]["question"]
                },
                "answer": {
                    "en": data["en"]["answer"],
                    "zh": data["zh"]["answer"],
                    language: data[language]["answer"]
                },
                "data_source": dataset
            }
            sample_list.append(sample)
    output_path = output_path.format(language)
    with open(output_path, "w") as fw:
        json.dump(fp=fw, obj=sample_list, indent=4, ensure_ascii=False)


common ../data/common/mling_common.json


UnicodeDecodeError: 'gbk' codec can't decode byte 0x80 in position 359: illegal multibyte sequence

In [ ]:
import os
import backoff
import openai
from tenacity import retry, stop_after_attempt, wait_random_exponential
import time
import numpy as np
from tqdm import tqdm, trange
from ipdb import set_trace
import json
import jsonlines

import argparse
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

openai.api_key = ""


def read_json(filename):
    with open(filename, "r", encoding="utf-8") as fr:
        data_pool = json.load(fr)

    return data_pool


def get_chatgpt_info(model_name: str, input_context: str, temperature: float) -> str:
    response = openai.ChatCompletion.create(
        model=model_name,
        messages=[
            {
                "role": "system",
                "content": "You are an excellent question responder.",
            },
            {
                "role": "user",
                "content": input_context,
            },
        ],
        temperature=temperature,
    )
    
    return response["choices"][0]["message"]["content"]


prompt_form = "icl"
temperature = 0.8
test_set = "test_1k"
model_name = "gpt-3.5-turbo"
template_path = "../prompt/selfaware/template.json"
prompt_path = "../prompt/selfaware/" + prompt_form + ".txt"
data_path = "../data/selfaware/" + test_set + ".json"
exp_path = "../exp/selfaware"
output_path = os.path.join(exp_path, "{}_{}_{}_T{}.json".format(model_name, prompt_form, test_set, temperature))

open(output_path, "w").close()

template_input = json.load(open(template_path))
prompt = open(prompt_path, "r").read()
    
data_list = read_json(data_path)
length = len(data_list)
if not os.path.exists(model_name):
    os.mkdir(model_name)
print("Model Name:", model_name, "Temperature:", temperature, "Prompt Form:", prompt_form)

for idx in range(length):
    question = data_list[idx]
        
    input_context = template_input["prompt_input"].format(prompt, question)
#     print(input_context)
    generated_text = get_chatgpt_info(model_name, input_context, temperature)

    data_list[idx]["generation"] = generated_text
    with jsonlines.open(output_path, mode="a") as writer:
        writer.write(data_list[idx])

In [6]:
import utils

input_path = "./../data/gsm8k/mling_gsm8k.json"
output_path = "./../data/gsm8k/mling_gsm8k.json"

lang_list = ["zh", "ja", "th", "fr"]

data_pool = utils.read_json(input_path)
for data in data_pool:
    answer = data["answer"]["en"].split("####")[-1]
    # print(answer)
    for lang in lang_list:
        data["answer"][lang] += "\n####" + answer
        # print(data["answer"][lang])

utils.write_json(output_path, data_pool)

In [8]:
from utils import jsonl2json

model = "gpt-3.5-turbo"
dataset = "gsm8k"
data_path = f"./../exp/{dataset}/inference/{model}_{dataset}_infer/generate.json"

jsonl2json(data_path, data_path)

In [ ]:
from utils import jsonl2json

model = "gpt-3.5-turbo"
dataset = "triviaqa"
data_path = f"./../exp/{dataset}/confidence/{model}_{dataset}_confidence/confidence.json"

jsonl2json(data_path, data_path)